# KeyFrame Extraction

In [13]:
import cv2
import os
import shutil

def clear_output_folder(folder):
    """ Clears all files in the given folder. """
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder)

# Global variables for cropping
cropping = False
x_start, y_start, x_end, y_end = 0, 0, 0, 0

def click_and_crop(event, x, y, flags, param):
    """ Mouse callback function for selecting the crop region. """
    global x_start, y_start, x_end, y_end, cropping, frame

    # Record the starting (x, y) coordinates on mouse click
    if event == cv2.EVENT_LBUTTONDOWN:
        x_start, y_start, x_end, y_end = x, y, x, y
        cropping = True

    # Record the ending (x, y) coordinates on mouse release
    elif event == cv2.EVENT_LBUTTONUP:
        x_end, y_end = x, y
        cropping = False

        # Draw a rectangle around the region of interest
        cv2.rectangle(frame, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
        cv2.imshow("frame", frame)

def extract_and_crop_frames(video_path, output_folder, n):
    global x_start, y_start, x_end, y_end, frame

    clear_output_folder(output_folder)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    if not cap.isOpened():
        print("Error opening video file")
        return

    ret, frame = cap.read()
    if not ret:
        print("Failed to read the video")
        cap.release()
        return

    cv2.namedWindow("frame")
    cv2.setMouseCallback("frame", click_and_crop)

    while True:
        # Display the image and wait for a keypress
        cv2.imshow("frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # Break from the loop after the 'c' key is pressed
        if key == ord("c"):
            break

    cv2.destroyAllWindows()

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Check if the frame is one of the Nth frames
        if frame_count % n == 0:
            # Crop the frame
            cropped_frame = frame[y_start:y_end, x_start:x_end]

            # # Resize the frame to 512dx512
            # resized_frame = cv2.resize(cropped_frame, (300,512))

            # Save the frame
            output_path = os.path.join(output_folder, f'frame_{frame_count}.jpg')
            cv2.imwrite(output_path, cropped_frame)

        frame_count += 1

    cap.release()

In [8]:
Actions = ["run"]
for action in Actions:
    video_path = 'Videos/'+action+'.mkv'
    output_folder = 'KeyFrames/'+action
    n = 8
    if (os.path.exists(video_path)):
        extract_and_crop_frames(video_path, output_folder, n)

In [26]:
Actions = ["idle"]
for action in Actions:
    video_path = 'Videos/'+action+'.mkv'
    output_folder = 'KeyFrames/'+action
    n = 90
    if (os.path.exists(video_path)):
        extract_and_crop_frames(video_path, output_folder, n)

In [28]:
Actions = ["jump2"]
for action in Actions:
    video_path = 'Videos/'+action+'.mkv'
    output_folder = 'KeyFrames/'+action
    n = 25
    if (os.path.exists(video_path)):
        extract_and_crop_frames(video_path, output_folder, n)

# Sprite Sheet Preparation

In [30]:
import re
def natural_sort_key(s):
    """
    Provides a natural sort key for sorting strings containing numbers.

    :param s: The string to be sorted.
    :return: A tuple containing parts of the string and extracted numbers.
    """
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]


def stack_frames_horizontally(folder_path):
    """
    Horizontally stacks all frames in the specified folder.

    :param folder_path: Path to the folder containing the frames.
    :return: Horizontally stacked image.
    """
    frames = []
    print(os.listdir(folder_path))
    for filename in sorted(os.listdir(folder_path),key=natural_sort_key):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            print(filename)
            img = cv2.imread(os.path.join(folder_path, filename))
            if img is not None:
                frames.append(img)

    # Check if there are any frames to concatenate
    if not frames:
        print("No frames found in the folder.")
        return None

    # Resize frames to the same height
    height = min(frame.shape[0] for frame in frames)
    resized_frames = [cv2.resize(frame, (int(frame.shape[1] * height / frame.shape[0]), height)) for frame in frames]

    # Concatenate horizontally
    stacked_image = cv2.hconcat(resized_frames)
    return stacked_image


Action = ["jump2"]

action = Action[0]

folder_path = 'KeyFrames/'+action  # Path to the folder containing the frames
stacked_image = stack_frames_horizontally(folder_path)

if stacked_image is not None:

    stacked_image = cv2.resize(stacked_image,(2048,512))

    cv2.imshow('Stacked Image', stacked_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cv2.imwrite('SpriteSheetTemplates/'+action+'.jpg', stacked_image)

['frame_0.jpg', 'frame_100.jpg', 'frame_25.jpg', 'frame_50.jpg', 'frame_75.jpg']
frame_0.jpg
frame_25.jpg
frame_50.jpg
frame_75.jpg
frame_100.jpg


# Experimental: Stack One-Row SpriteSheets

In [17]:
def stack_frames_vertically(folder_path):
    """
    Vertically stacks all frames in the specified folder.

    :param folder_path: Path to the folder containing the frames.
    :return: Vertically stacked image.
    """
    frames = []
    for filename in sorted(os.listdir(folder_path), key=natural_sort_key):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = cv2.imread(os.path.join(folder_path, filename))
            if img is not None:
                frames.append(img)

    # Check if there are any frames to concatenate
    if not frames:
        print("No frames found in the folder.")
        return None

    # Resize frames to the same width
    width = min(frame.shape[1] for frame in frames)
    resized_frames = [cv2.resize(frame, (width, int(frame.shape[0] * width / frame.shape[1]))) for frame in frames]

    # Concatenate vertically
    stacked_image = cv2.vconcat(resized_frames)
    return stacked_image

# Example usage
folder_path = 'SpriteSheetTemplates'  # Replace with your folder path
stacked_image = stack_frames_vertically(folder_path)

if stacked_image is not None:
    cv2.imshow('Stacked Image', stacked_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cv2.imwrite('AllInOne.jpg', stacked_image)